In [1]:
#import libraries 
from simplesam import Reader 
import pandas as pd
#import argparse

As we performed paired end sequencing the sam file includes the read from the end of each insert .Thus, to find the coordinates of each read (insert) we need only to consider every second entry
self.pos returns the left most point of the read and self.tlen returns the length of the entire read.Also note that SAM files use a 1 based coordinate system (unlike the 0 based coordinate system) therefore this must be converted.Therefore if self is the left read of a sequence the coordinates of the read are given by (x.pos-1,x.pos+x.tlen-1).

In [2]:
#make an argument parser 
parser = argparse.ArgumentParser(
    description='determine reads which contain viral DNA')

parser.add_argument('--sam', help = 'sam file describing simulated reads', required = True)
parser.add_argument('--host_ints', help = 'csv file describing the location of viral DNA in the human host', required = True)

NameError: name 'argparse' is not defined

In [ ]:
#read in the sam file 
in_file = open('paired_dat.sam','r')
in_sam = Reader(in_file)

#read in our integration file 
integration_file = pd.read_csv('host_integrations.csv',header=0,sep='\t')

#make a df of the entries in the sam file 
df = pd.read_csv('paired_dat.sam',header=None) 
num_reads = len(df[~df[0].str.contains('@')])
num_inserts = int(num_reads/2)

#list of insert IDs
read_id = []

#list of insert coordinates in the original fasta sequence
read_coord = []

for i in range(0,num_inserts): 
    x = next(in_sam)
    y = next(in_sam)
    read_id.append(x.qname)
    read_coord.append((x.pos-1,x.pos+x.tlen-1))

In [3]:
num_inserts

4320610

In [3]:
def checkOverlap(coordA,coordB):
    """Tells us whether coordA overlaps with coordB"""
    status = False 
    A1, A2 = coordA
    B1, B2 = coordB 
    if A2>B1 and A1<B1:
        status = True 
    return status 

def overlapLength(coordA,coordB):
    """Tells us the length of the overlap betweeen coordA and coordB"""
    A1, A2 = coordA
    B1, B2 = coordB
    overlap = 0
    #isolate where overlaps occur
    if A2>B1 and A1<B1:
        #if overlap is at the left end of B
        if A1<B1: 
            overlap = A2-B1
        #if overlap is at the right end of B
        if A2>B2: 
            overlap = B2-A1
        #if overlap is within B 
        else: 
            overlap = A2-A1
    return overlap 

In [4]:
#list of integration coordinates in host 
int_coord = []

for i in range(len(integration_file)):
    c1 = integration_file["Start point"][i]
    c2 = integration_file["Stop point"][i]
    int_coord.append((c1,c2))
    
#list of overlap statuses 
overlap_status = []

#list of overlap lengths 
overlap_len = []

for i in range(num_inserts):
    overlap = 0
    status = False
    for j in range(len(integration_file)):
        status = checkOverlap(read_coord[i],int_coord[j])
        if status == True:
            overlap = overlapLength(read_coord[i],int_coord[j])
            break
    overlap_status.append(status)
    overlap_len.append(overlap)
            

In [5]:
#make and save a dataframe to save the output
header = ["read_id","contains_virus","virus_length"]
results = pd.DataFrame({"read_id":read_id,
                        "contains_virus":overlap_status,
                        "virus_length":overlap_len})

In [6]:
results.to_csv("read_integrations.csv",sep='\t')

In [ ]:
results.to_csv(index=False)

In [ ]:
results.to_csv('trial.csv',sep='\t')

In [3]:
#read in the sam file 
in_file = open('test_dat.sam','r')
in_sam = Reader(in_file)

#read in our integration file 
integration_file = pd.read_csv('host_integrations.csv',header=0,sep='\t')

In [6]:
x = next(in_sam)

In [7]:
x.qname

'host-280'

In [8]:
x.seq

'ATCAAAAGTTTACCCCCCCCCCCCCCCCCCCCCCTCCGCCTCCGCCGATACTGGTCGCACCCCATTATTTTAGTTCGTTCGATGCCGGGAAGTTCCAAAGCCTATCACGAGGAAGATCTCAGAAAGCGGCTTAGACTACAAGCCCCTCTG'

In [17]:
y.seq

'TACGATCACCTTATCGTTTACCTGGGTTTGTGTGGGAGATTACGGAAAAAAGGTAGGTGTCGCTTAAACCGAATCGTCACCCGCGTCTGTCGCGCCACAGGCGTGTGTTAATCCCTACGAGAGAIDCLMJYQAAAATCCGGGTGATGGAA'

In [15]:
for i in range(0,1):
#print(x.qname)
    x = next(in_sam)
x.qname    

'host-110'

In [17]:
x.seq

'ACCTATGATGATCTGCCGCCAAGGTCCGCCTTATGTTAATGGGACCGTTAGGTACTACTGGGGCTGGATCCCCATACAGCTAAAGGTATCAGGTAGGTCTTTAGGACAAAGTCCGAGAGATGTTCGGGCACCAGGACGTTAGTCATGGGA'

In [20]:
q = next(in_sam)

In [22]:
q.seq

'GTTTATCCCGCTCTGCCGCTAAGCGAAGGATTCCGTTCGAAACGCTTCAGTAGAAATCACCGAACGACGTTTGCCCACCGCGACAGCCACCTNNNNNNNNNNNACTCACCCCCAGGCTGCGCTGACCCAAAAGCCTGTGGCGAAACCCTT'